In [41]:
from config import COMMON_CONFIG

In [42]:
!pip install matplotlib -q
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

!pip install pandas -q
import pandas as pd
# Хак чтобы работало на Anaconda под MacOs 10.15.7 Catalina
pd.DataFrame.iteritems = pd.DataFrame.items

In [43]:
#!pip install git+https://github.com/kernc/backtesting.py.git -q
!pip install backtesting -q

from backtesting import Strategy
from backtesting_utils import run_backtest
from backtesting_utils import convert_data_to_bt_format

In [44]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

import random
import numpy as np

def seed_everything(seed: int = COMMON_CONFIG.SEED) -> None:
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    print(f"Using {seed} seed")

# Зафиксируем SEED
seed_everything(COMMON_CONFIG.SEED)

Using 777 seed


In [45]:
from file_utils import read_data_from_file
from data_utils import train_test_split

df = read_data_from_file(COMMON_CONFIG.SYMBOL, COMMON_CONFIG.TIMEFRAME)
df

,open,high,low,close,volume
date,,,,,
2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336
2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516
2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623
2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433
2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747
...,...,...,...,...,...
2025-01-19 09:00:00,3221.90,3233.77,3143.02,3226.00,105144.51170
2025-01-19 10:00:00,3226.01,3226.89,3172.97,3174.70,51453.00680
2025-01-19 11:00:00,3174.70,3188.90,3130.48,3149.97,50955.62760


In [46]:
!pip install tsfresh -q

from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh.utilities.dataframe_functions import impute

In [47]:
df_for_tsfresh = df.copy()
df_for_tsfresh

,open,high,low,close,volume
date,,,,,
2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336
2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516
2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623
2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433
2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747
...,...,...,...,...,...
2025-01-19 09:00:00,3221.90,3233.77,3143.02,3226.00,105144.51170
2025-01-19 10:00:00,3226.01,3226.89,3172.97,3174.70,51453.00680
2025-01-19 11:00:00,3174.70,3188.90,3130.48,3149.97,50955.62760


In [48]:
df_melted = pd.DataFrame()
df_melted["timestamp"] = df_for_tsfresh.index
df_melted["close"] = df_for_tsfresh['close'].values
df_melted['symbol'] = COMMON_CONFIG.SYMBOL.replace('/', '')
df_melted.tail(3)

,timestamp,close,symbol
44257,2025-01-19 11:00:00,3149.97,ETHUSDT
44258,2025-01-19 12:00:00,3205.90,ETHUSDT
44259,2025-01-19 13:00:00,3220.79,ETHUSDT


In [49]:
df_rolled = roll_time_series(df_melted, column_id="symbol", column_sort="timestamp",
                             min_timeshift=COMMON_CONFIG.TS_FRESH_MIN_WINDOW_SIZE,
                             max_timeshift=COMMON_CONFIG.TS_FRESH_MAX_WINDOW_SIZE)
df_rolled

Rolling: 100%|██████████| 40/40 [01:35<00:00,  2.39s/it]


,timestamp,close,symbol,id
0,2020-01-01 00:00:00,128.87,ETHUSDT,"(ETHUSDT, 2020-01-01 05:00:00)"
1,2020-01-01 01:00:00,130.64,ETHUSDT,"(ETHUSDT, 2020-01-01 05:00:00)"
2,2020-01-01 02:00:00,130.85,ETHUSDT,"(ETHUSDT, 2020-01-01 05:00:00)"
3,2020-01-01 03:00:00,130.20,ETHUSDT,"(ETHUSDT, 2020-01-01 05:00:00)"
4,2020-01-01 04:00:00,130.20,ETHUSDT,"(ETHUSDT, 2020-01-01 05:00:00)"
...,...,...,...,...
2698010,2025-01-19 09:00:00,3226.00,ETHUSDT,"(ETHUSDT, 2025-01-19 13:00:00)"
2698011,2025-01-19 10:00:00,3174.70,ETHUSDT,"(ETHUSDT, 2025-01-19 13:00:00)"
2698012,2025-01-19 11:00:00,3149.97,ETHUSDT,"(ETHUSDT, 2025-01-19 13:00:00)"
2698013,2025-01-19 12:00:00,3205.90,ETHUSDT,"(ETHUSDT, 2025-01-19 13:00:00)"


In [50]:
X = extract_features(df_rolled.drop("symbol", axis=1), 
                     column_id="id", column_sort="timestamp", column_value="close", 
                     impute_function=impute, show_warnings=False)
X

Feature Extraction:   0%|          | 0/40 [01:36<?, ?it/s]
Process SpawnPoolWorker-37:
Traceback (most recent call last):
  File "/Users/w3dip/opt/anaconda3/envs/code3_8/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/w3dip/opt/anaconda3/envs/code3_8/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/w3dip/opt/anaconda3/envs/code3_8/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/w3dip/opt/anaconda3/envs/code3_8/lib/python3.8/site-packages/tsfresh/utilities/distribution.py", line 43, in _function_with_partly_reduce
    results = list(itertools.chain.from_iterable(results))
  File "/Users/w3dip/opt/anaconda3/envs/code3_8/lib/python3.8/site-packages/tsfresh/utilities/distribution.py", line 42, in <genexpr>
    results = (map_function(chunk, **kwargs) for chunk in chunk_list)
  File "/Users/w3dip/opt/anac

KeyboardInterrupt: 

In [23]:
X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
X.index.name = "last_timestamp"
X

,close__variance_larger_than_standard_deviation,close__has_duplicate_max,close__has_duplicate_min,close__has_duplicate,close__sum_values,close__abs_energy,close__mean_abs_change,close__mean_change,close__mean_second_derivative_central,close__median,...,close__fourier_entropy__bins_5,close__fourier_entropy__bins_10,close__fourier_entropy__bins_100,close__permutation_entropy__dimension_3__tau_1,close__permutation_entropy__dimension_4__tau_1,close__permutation_entropy__dimension_5__tau_1,close__permutation_entropy__dimension_6__tau_1,close__permutation_entropy__dimension_7__tau_1,close__query_similarity_count__query_None__threshold_0.0,close__mean_n_absolute_max__number_of_maxima_7
last_timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 05:00:00,0.0,0.0,0.0,1.0,781.06,1.016782e+05,0.546000,0.286000,-0.208750,130.25,...,1.039721,1.386294,1.386294,1.039721,1.098612,0.693147,-0.000000,3.982128,0.0,324.541429
2020-01-01 06:00:00,0.0,0.0,0.0,1.0,911.50,1.186928e+05,0.478333,0.261667,-0.163000,130.30,...,1.386294,1.386294,1.386294,0.950271,1.386294,1.098612,0.693147,-0.000000,0.0,324.541429
2020-01-01 07:00:00,0.0,0.0,0.0,1.0,1041.74,1.356552e+05,0.438571,0.195714,-0.164167,130.27,...,0.950271,0.950271,1.609438,1.011404,1.332179,1.386294,1.098612,0.693147,0.0,130.410000
2020-01-01 08:00:00,0.0,0.0,0.0,1.0,1172.10,1.526490e+05,0.398750,0.186250,-0.117857,130.30,...,1.332179,1.609438,1.609438,1.078992,1.560710,1.609438,1.386294,1.098612,0.0,130.432857
2020-01-01 09:00:00,0.0,0.0,0.0,1.0,1302.27,1.695932e+05,0.375556,0.144444,-0.122500,130.27,...,1.011404,1.329661,1.791759,1.082196,1.747868,1.791759,1.609438,1.386294,0.0,130.432857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-21 07:00:00,1.0,0.0,0.0,0.0,119243.90,2.331791e+08,11.928000,-0.164333,-0.369576,1943.89,...,0.283936,0.424254,1.611462,1.722432,2.939295,3.722446,3.916987,4.007333,0.0,2020.040000
2021-02-21 08:00:00,1.0,0.0,0.0,0.0,119258.30,2.332347e+08,12.071500,-0.020833,0.146864,1943.89,...,0.424254,0.424254,1.505145,1.734833,2.954175,3.698125,3.916987,4.007333,0.0,2020.040000
2021-02-21 09:00:00,1.0,0.0,0.0,0.0,119260.18,2.332419e+08,12.008167,-0.084167,0.087034,1943.89,...,0.424254,0.424254,1.566743,1.724862,2.921252,3.673804,3.892232,4.007333,0.0,2020.040000


In [24]:
y = df_melted.set_index("timestamp").sort_index().close.shift(-1)
y.tail(3)

timestamp
2021-02-21 09:00:00    1971.19
2021-02-21 10:00:00    1961.62
2021-02-21 11:00:00        NaN
Name: close, dtype: float64

In [25]:
y = y[y.index.isin(X.index)]
X = X[X.index.isin(y.index)]

In [26]:
X

,close__variance_larger_than_standard_deviation,close__has_duplicate_max,close__has_duplicate_min,close__has_duplicate,close__sum_values,close__abs_energy,close__mean_abs_change,close__mean_change,close__mean_second_derivative_central,close__median,...,close__fourier_entropy__bins_5,close__fourier_entropy__bins_10,close__fourier_entropy__bins_100,close__permutation_entropy__dimension_3__tau_1,close__permutation_entropy__dimension_4__tau_1,close__permutation_entropy__dimension_5__tau_1,close__permutation_entropy__dimension_6__tau_1,close__permutation_entropy__dimension_7__tau_1,close__query_similarity_count__query_None__threshold_0.0,close__mean_n_absolute_max__number_of_maxima_7
last_timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 05:00:00,0.0,0.0,0.0,1.0,781.06,1.016782e+05,0.546000,0.286000,-0.208750,130.25,...,1.039721,1.386294,1.386294,1.039721,1.098612,0.693147,-0.000000,3.982128,0.0,324.541429
2020-01-01 06:00:00,0.0,0.0,0.0,1.0,911.50,1.186928e+05,0.478333,0.261667,-0.163000,130.30,...,1.386294,1.386294,1.386294,0.950271,1.386294,1.098612,0.693147,-0.000000,0.0,324.541429
2020-01-01 07:00:00,0.0,0.0,0.0,1.0,1041.74,1.356552e+05,0.438571,0.195714,-0.164167,130.27,...,0.950271,0.950271,1.609438,1.011404,1.332179,1.386294,1.098612,0.693147,0.0,130.410000
2020-01-01 08:00:00,0.0,0.0,0.0,1.0,1172.10,1.526490e+05,0.398750,0.186250,-0.117857,130.30,...,1.332179,1.609438,1.609438,1.078992,1.560710,1.609438,1.386294,1.098612,0.0,130.432857
2020-01-01 09:00:00,0.0,0.0,0.0,1.0,1302.27,1.695932e+05,0.375556,0.144444,-0.122500,130.27,...,1.011404,1.329661,1.791759,1.082196,1.747868,1.791759,1.609438,1.386294,0.0,130.432857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-21 07:00:00,1.0,0.0,0.0,0.0,119243.90,2.331791e+08,11.928000,-0.164333,-0.369576,1943.89,...,0.283936,0.424254,1.611462,1.722432,2.939295,3.722446,3.916987,4.007333,0.0,2020.040000
2021-02-21 08:00:00,1.0,0.0,0.0,0.0,119258.30,2.332347e+08,12.071500,-0.020833,0.146864,1943.89,...,0.424254,0.424254,1.505145,1.734833,2.954175,3.698125,3.916987,4.007333,0.0,2020.040000
2021-02-21 09:00:00,1.0,0.0,0.0,0.0,119260.18,2.332419e+08,12.008167,-0.084167,0.087034,1943.89,...,0.424254,0.424254,1.566743,1.724862,2.921252,3.673804,3.892232,4.007333,0.0,2020.040000


In [29]:
y

timestamp
2020-01-01 05:00:00     130.44
2020-01-01 06:00:00     130.24
2020-01-01 07:00:00     130.36
2020-01-01 08:00:00     130.17
2020-01-01 09:00:00     130.70
                        ...   
2021-02-21 07:00:00    1936.95
2021-02-21 08:00:00    1940.08
2021-02-21 09:00:00    1971.19
2021-02-21 10:00:00    1961.62
2021-02-21 11:00:00        NaN
Name: close, Length: 9995, dtype: float64

In [30]:
X_train, X_test = train_test_split(X)

Train size: 1224, Test size: 8771


In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train, y_val, y_test = train_test_split(y)

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
X_train_selected = select_features(X_train, y_train)
X_train_selected

In [ ]:
#X_val_selected = X_val[X_train_selected.columns]
#X_val_selected

In [ ]:
X_test_selected = X_test[X_train_selected.columns]
X_test_selected

In [ ]:
from sklearn.utils import all_estimators
from sklearn import base
import sklearn.metrics as metrics 

from functions import init_regressors,load_data,get_best_regressor,view_performance

# Choose whether to look for a best regression model, or use a pre-saved one from a previous run, alternatively you can choose your own regression method.
init = 0                  # do not (0) or (1) calculate the best regressor
best_regressor = ''       # you can choose a regressor here

# Choose to see the best_regressor calculations in verbose mode
verbose = 0

ticker = COMMON_CONFIG.SYMBOL.replace('/', '')

test_start = X.index.max() - pd.DateOffset(months=int(1))
test_start

In [ ]:
if init == 1:
    init_regressors(ticker)

In [ ]:
try:
    with open(f'regressor_min_mae_{ticker}.csv') as f:
        best_regressor=f.read()
    print ('Last saved regression method:',best_regressor)
except:
    pass

In [ ]:
# For cleaner output we suppress warnings that occur during testing of the estimators
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

simplefilter("ignore", category=FutureWarning)
simplefilter("ignore", category=RuntimeWarning)
simplefilter("ignore", category=ConvergenceWarning)
simplefilter("ignore", category=UserWarning)

# Check and proceed if init=1 or no best_regressor was found
if (best_regressor == '') | init == 1:

    # Create a list for regressors with too bad performance
    new_removed_regressors = []
    removed_regressors = pd.DataFrame()

    # Calculate all the estimator metrics
    MAE, MSE, RMSE, R2, removed_regressors, new_removed_regressors = get_best_regressor(X_train_selected, X_test,y_train, y, test_start, ticker, verbose)

    # Write the results to this file
    with open(f'regressor_min_mae_{ticker}.csv', 'w') as f:
        f.write(min(MAE, key=MAE.get))

    # Define the best estimator by chosing the one with the smallest MAE
    best_regressor = min(MAE, key=MAE.get)

    # Create a dataframe with the excluded estimators and write it to a file, so we can exclude them in future runs 
    if new_removed_regressors != 'None':
        new_removed_regressors = pd.DataFrame(new_removed_regressors, columns=['name'])
        removed_regressors = pd.concat([removed_regressors, new_removed_regressors])
        removed_regressors.to_csv(f'removed_regressors_{ticker}.csv', 'a', index=False)

# Load the estimator model and store its name
estimators = all_estimators()
for est_name, estimator in estimators:
    if str(best_regressor) == est_name:
        best_regressor = estimator()
        name = est_name

# Print out the five best estimators
df_best_regressors = pd.DataFrame.from_dict({'MAE': MAE, 'RMSE': RMSE, 'R2_score': R2})
df_best_regressors = df_best_regressors.sort_values(by="MAE", ascending=True).head().reset_index()
df_styled = df_best_regressors.style.apply(lambda x: ['background-color: lightgreen' if x.name == 0 else '' for i in x],
                                           axis=1)

display(df_styled)

In [ ]:
# Load the estimator model and store its name
estimators=all_estimators()
for est_name, estimator in estimators:    
    if str(best_regressor)==est_name:
        best_regressor=estimator()
        name=est_name

In [ ]:
best_regressor.fit(X_train_selected, y_train)

In [ ]:
y_pred = pd.DataFrame(best_regressor.predict(X_test_selected), index=X_test_selected.index, columns=['predicted_close'])
y_pred

In [ ]:
y_test.tail(3)

In [ ]:
y_pred.tail(3)

In [ ]:
from plots import plot

all_predictions = y_pred#['predicted_close']
all_targets = y_test#['close']
plot(all_predictions, all_targets, "ML model Predictions vs. Actual")

In [ ]:
df_best_regressors